In [ ]:
from osgeo import gdal
import os
import glob
import numpy as np

In [ ]:
def read_tiff(input_file):
    """
    读取影像
    :param input_file:输入影像
    :return:波段数据，仿射变换参数，投影信息、行数、列数、波段数
    """

    dataset = gdal.Open(input_file)
    rows = dataset.RasterYSize
    cols = dataset.RasterXSize

    geo = dataset.GetGeoTransform()
    proj = dataset.GetProjection()
    couts = dataset.RasterCount
    # if (satellite == 'wv3' or satellite == 'wv2') and couts == 8:
    #     bands = [2,3,5,7] # b, g, r, nir
    #     array_data = np.zeros((4,rows,cols))

    #     for i in range(len(bands)):
    #         band = dataset.GetRasterBand(bands[i])
    #         array_data[i,:,:] = band.ReadAsArray()
    #     couts = 4
    # else:
    array_data = dataset.ReadAsArray()

    return array_data,geo,proj,rows,cols,couts

In [ ]:
input_data_path = r'D:\YourPath'
input_pan_path = glob.glob(os.path.join(input_data_path, "pan.tif"))#r'./pan.tif'
array_data,geo,proj,rows,cols,couts = read_tiff(input_pan_path[0])
print(array_data.shape)

In [ ]:
from PIL import Image
im = Image.open(r'D:\YourPath\lr_red.tif')
print(im.format, im.size, im.mode)
im.show()

In [ ]:
# -*- coding:utf-8 -*-
import numpy as np
from osgeo import gdal
import os
import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

#修改路径
tifDir = r"D:\YourPath"  #tif路径 单波段
outtif = r"D:\YourPath\mul.tif"


NP2GDAL_CONVERSION = {
  "uint8": 1,
  "int8": 1,
  "uint16": 2,
  "int16": 3,
  "uint32": 4,
  "int32": 5,
  "float32": 6,
  "float64": 7,
  "complex64": 10,
  "complex128": 11,
}
tifs = [i for i in os.listdir(tifDir) if i.endswith(".tif")]
print(tifs)
#获取投影波段数等信息
bandsNum = len(tifs)
dataset = gdal.Open(os.path.join(tifDir,tifs[0]))
projinfo = dataset.GetProjection()
geotransform = dataset.GetGeoTransform()
cols,rows=dataset.RasterXSize,dataset.RasterYSize
datatype=dataset.GetRasterBand(1).ReadAsArray(0,0,1,1).dtype.name
print(cols, rows, datatype)
gdaltype=NP2GDAL_CONVERSION[datatype]
dataset=None
#创建目标文件
format = "GTiff" #tif格式
format = "ENVI"  # ENVI格式
driver = gdal.GetDriverByName(format)
dst_ds = driver.Create(outtif,cols, rows, bandsNum, gdaltype)
dst_ds.SetGeoTransform(geotransform)
dst_ds.SetProjection(projinfo)
#写入文件
info = set()
for k in range(bandsNum):
    ds = gdal.Open(os.path.join(tifDir,tifs[k]))
    X,Y = ds.RasterXSize,ds.RasterYSize
    info.add("%s,%s"%(X,Y))
    if(len(info) != 1):
        dst_ds = None
        ds = None
        print("%s 列数行数不一样：%s,%s"%(tifs[k],X,Y))
        raise Exception("有影像行列数不一致")
    data = ds.GetRasterBand(1).ReadAsArray()    ##读取第一波段
    ds = None
    dst_ds.GetRasterBand(k+1).WriteArray(data)
    dst_ds.GetRasterBand(k+1).SetDescription("Band_%s"%(k+1))

    print("波段 %s ==> 文件 %s"%(k+1,tifs[k]))
dst_ds = None

In [ ]:
import glob
dirIn = r"D:\YourPath"
LRList = glob.glob(os.path.join(dirIn, '*.tif'))
print(LRList)